In [1]:
import pandas as pd
import numpy as np
import re
import os
from glob import glob
PATH = '../Tracking_All'
EXT = "*.csv"

## data -> [X,Y,area,major,minor]

In [3]:
X = np.load('../npy_files/features_All32.npy')
y = np.load('../npy_files/labels_All32.npy')
types = np.load('../npy_files/celltypes_All32.npy')
y_letter = np.array([well[:1] for well in y])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labs = le.fit_transform(y)
labs_by_letter = le.fit_transform(y_letter)
print(len(X),len(y))
X_train, y_train = X, labs

4600 4600


### Ellipticity

In [45]:
def spheroid(long_sq,short_sq):
    return np.sqrt((long_sq-short_sq)/long_sq)
def elip(majorminor):
    maj_sq = majorminor[0]**2
    min_sq = majorminor[1]**2
    return spheroid(maj_sq,min_sq) if maj_sq>min_sq else spheroid(min_sq,maj_sq)

ellipticity = np.apply_along_axis(elip, 2, X_train[:,:,3:5])
ellipticity.shape

(4600, 32)

## step size

In [46]:
d = np.diff(X_train[:,:,:2],axis=1,prepend=0)
step_size = np.sqrt(np.power(d,2).sum(axis=2))
step_size.shape

(4600, 32)

## angle of movement

In [67]:
def getAngleRad(diffs):
  return np.arctan2(diffs[1], diffs[0])

d = np.diff(X_train[:,:,:2],axis=1,prepend=0)
angles_of_step = np.apply_along_axis(getAngleRad, 2, d)
angles_of_step.shape

(4600, 32)

In [68]:
angles_of_step[0]

array([ 0.        , -3.00987842, -1.12923543, -0.40969072,  1.20597319,
       -0.02276494,  1.52632677, -0.48320219,  0.83398422,  1.63549678,
       -0.88792091,  2.85377003, -0.17551316,  2.97570814,  2.69177388,
        1.26294912, -2.84605946,  1.08439393, -0.68940841, -1.56675927,
       -2.4120934 , -0.09711885,  0.18281433, -3.06148952,  3.02897935,
        1.41443126, -3.06489046,  2.29363454,  1.02146026,  2.454647  ,
       -0.71825304, -2.98287973])

## angle from center

In [62]:
def getAngleRadZero(xy):
  return np.arctan2(xy[1]-0, xy[0]-0)

angles_from_center = np.apply_along_axis(getAngleRad, 2, d)
angles_from_center.shape

(4600, 32)

In [66]:
angles_from_center[0]

array([ 0.        , -3.00987842, -1.12923543, -0.40969072,  1.20597319,
       -0.02276494,  1.52632677, -0.48320219,  0.83398422,  1.63549678,
       -0.88792091,  2.85377003, -0.17551316,  2.97570814,  2.69177388,
        1.26294912, -2.84605946,  1.08439393, -0.68940841, -1.56675927,
       -2.4120934 , -0.09711885,  0.18281433, -3.06148952,  3.02897935,
        1.41443126, -3.06489046,  2.29363454,  1.02146026,  2.454647  ,
       -0.71825304, -2.98287973])